<a href="https://colab.research.google.com/github/adityamukherjee42/CNN-from-Scratch/blob/main/CNN_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [2]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
num_workers=2
batch_size=20
valid_size=0.2

transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_data=datasets.CIFAR10('data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,download=True, transform=transform)

num_train=int(len(train_data))
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)


classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [4]:
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1=nn.Conv2d(3,16,3,padding=1)
    self.conv2=nn.Conv2d(16,32,3,padding=1)
    self.conv3=nn.Conv2d(32,64,3,padding=1)

    self.pool=nn.MaxPool2d(2,2)

    self.fc1=nn.Linear(64*4*4,512)
    self.fc2=nn.Linear(512,10)

    self.Dropout=nn.Dropout(0.2)
  
  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=self.pool(F.relu(self.conv3(x)))

    x=x.view(-1,64*4*4)

    x=self.Dropout(x)

    x=F.relu(self.fc1(x))

    x=self.Dropout(x)

    x=self.fc2(x)

    return x

model = Net()
print(model)

if train_on_gpu:
    model.cuda()





Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (Dropout): Dropout(p=0.2, inplace=False)
)


In [6]:
criterion=nn.CrossEntropyLoss()
import torch.optim as optim
optimizer=optim.SGD(model.parameters(), lr=0.01)

In [7]:
epochs=30
valid_loss_min=np.Inf
for i in range(1,epochs+1):
  train_loss=0
  valid_loss=0

  model.train()
  for data,target in train_loader:
    if train_on_gpu:
      data, target = data.cuda(), target.cuda()
    optimizer.zero_grad()
    output=model(data)
    loss=criterion(output,target)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)

  model.eval()
  for data,target in valid_loader:
    if train_on_gpu:
      data, target = data.cuda(), target.cuda()
    optimizer.zero_grad()
    output=model(data)
    loss=criterion(output,target)
    loss.backward()
    optimizer.step()
    valid_loss += loss.item()*data.size(0)
  
  valid_loss=valid_loss/len(train_loader.sampler)
  train_loss=train_loss/len(train_loader.sampler)
  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(i, train_loss, valid_loss))
  if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss




Epoch: 1 	Training Loss: 2.125424 	Validation Loss: 0.439107
Validation loss decreased (inf --> 0.439107).  Saving model ...
Epoch: 2 	Training Loss: 1.578004 	Validation Loss: 0.353555
Validation loss decreased (0.439107 --> 0.353555).  Saving model ...
Epoch: 3 	Training Loss: 1.374569 	Validation Loss: 0.311877
Validation loss decreased (0.353555 --> 0.311877).  Saving model ...
Epoch: 4 	Training Loss: 1.236832 	Validation Loss: 0.278544
Validation loss decreased (0.311877 --> 0.278544).  Saving model ...
Epoch: 5 	Training Loss: 1.130236 	Validation Loss: 0.249727
Validation loss decreased (0.278544 --> 0.249727).  Saving model ...
Epoch: 6 	Training Loss: 1.035272 	Validation Loss: 0.227287
Validation loss decreased (0.249727 --> 0.227287).  Saving model ...
Epoch: 7 	Training Loss: 0.965668 	Validation Loss: 0.210148
Validation loss decreased (0.227287 --> 0.210148).  Saving model ...
Epoch: 8 	Training Loss: 0.903749 	Validation Loss: 0.193364
Validation loss decreased (0.21014

In [8]:

test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
for data, target in test_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    output = model(data)
    loss = criterion(output, target) 
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)    
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.973800

Test Accuracy of airplane: 81% (818/1000)
Test Accuracy of automobile: 87% (870/1000)
Test Accuracy of  bird: 67% (671/1000)
Test Accuracy of   cat: 61% (610/1000)
Test Accuracy of  deer: 72% (725/1000)
Test Accuracy of   dog: 63% (636/1000)
Test Accuracy of  frog: 82% (824/1000)
Test Accuracy of horse: 84% (845/1000)
Test Accuracy of  ship: 87% (871/1000)
Test Accuracy of truck: 85% (855/1000)

Test Accuracy (Overall): 77% (7725/10000)
